# Pytest Fixtures

## What You'll Learn
- What are fixtures and why use them?
- Creating basic fixtures
- Using fixtures in tests
- Fixture scopes (function, class, module, session)
- Setup and teardown with fixtures
- Fixture best practices

---

## The Problem: Repetitive Setup Code

Without fixtures, you end up repeating setup code in every test.

### Example: Testing a Database Class

In [ ]:
# database.py
class Database:
    """Simple database class."""
    
    def __init__(self):
        self.data = {}
    
    def add(self, key, value):
        """Add item to database."""
        self.data[key] = value
    
    def get(self, key):
        """Get item from database."""
        return self.data.get(key)
    
    def delete(self, key):
        """Delete item from database."""
        if key in self.data:
            del self.data[key]
    
    def count(self):
        """Return number of items."""
        return len(self.data)

### Without Fixtures (Repetitive Code)

In [ ]:
# test_database_without_fixtures.py
from database import Database

def test_add_item():
    db = Database()  # Create database
    db.add("name", "Alice")
    assert db.get("name") == "Alice"

def test_get_item():
    db = Database()  # Create database again
    db.add("age", 25)
    assert db.get("age") == 25

def test_delete_item():
    db = Database()  # Create database again
    db.add("city", "NYC")
    db.delete("city")
    assert db.get("city") is None

def test_count():
    db = Database()  # Create database again
    db.add("x", 1)
    db.add("y", 2)
    assert db.count() == 2

**Problems:**
- ❌ Repeating `db = Database()` in every test
- ❌ Hard to maintain - if setup changes, update everywhere
- ❌ Verbose and cluttered

**Solution:** Fixtures! 🎉

---

## What Are Fixtures?

**Fixtures** are functions that provide test data or setup/teardown code. They:
- ✅ Set up test data before tests run
- ✅ Can be reused across multiple tests
- ✅ Clean up after tests (if needed)
- ✅ Make tests cleaner and more maintainable

Think of fixtures as **"test ingredients"** - you prepare them once and use them in multiple tests.

---

## Creating Your First Fixture

### Step 1: Define a Fixture

In [ ]:
import pytest
from database import Database

@pytest.fixture
def db():
    """Create a database for testing."""
    database = Database()
    return database

**What happened:**
- `@pytest.fixture` decorator makes this a fixture
- Function name (`db`) is how you'll use it in tests
- Returns the object you want to use in tests

### Step 2: Use the Fixture in Tests

In [ ]:
def test_add_item(db):  # ← Notice the parameter name matches fixture name
    """Test adding an item."""
    db.add("name", "Alice")
    assert db.get("name") == "Alice"

def test_get_item(db):
    """Test getting an item."""
    db.add("age", 25)
    assert db.get("age") == 25

def test_delete_item(db):
    """Test deleting an item."""
    db.add("city", "NYC")
    db.delete("city")
    assert db.get("city") is None

def test_count(db):
    """Test counting items."""
    db.add("x", 1)
    db.add("y", 2)
    assert db.count() == 2

**Magic! ✨**
- Just add `db` as a parameter to your test
- Pytest automatically calls the `db()` fixture and passes the result
- Each test gets a fresh database
- No more repetition!

---

## Fixture with Initial Data

Fixtures can do more than just create objects - they can populate them with test data.

In [ ]:
@pytest.fixture
def db_with_users():
    """Create a database with sample users."""
    database = Database()
    database.add("user1", {"name": "Alice", "age": 25})
    database.add("user2", {"name": "Bob", "age": 30})
    database.add("user3", {"name": "Charlie", "age": 35})
    return database

def test_get_user(db_with_users):
    """Test retrieving a user."""
    user = db_with_users.get("user1")
    assert user["name"] == "Alice"
    assert user["age"] == 25

def test_count_users(db_with_users):
    """Test counting users."""
    assert db_with_users.count() == 3

def test_delete_user(db_with_users):
    """Test deleting a user."""
    db_with_users.delete("user2")
    assert db_with_users.count() == 2
    assert db_with_users.get("user2") is None

---

## Multiple Fixtures in One Test

You can use multiple fixtures in a single test!

In [ ]:
@pytest.fixture
def sample_user():
    """Create a sample user."""
    return {"name": "Alice", "email": "alice@example.com", "age": 25}

@pytest.fixture
def empty_db():
    """Create an empty database."""
    return Database()

def test_add_user_to_db(empty_db, sample_user):
    """Test adding a user to the database."""
    empty_db.add("user1", sample_user)
    
    retrieved_user = empty_db.get("user1")
    assert retrieved_user["name"] == "Alice"
    assert retrieved_user["email"] == "alice@example.com"

### Real-World Example: File Handling

In [ ]:
import os

@pytest.fixture
def temp_file():
    """Create a temporary file and clean it up after test."""
    filename = "test_data.txt"
    
    # Setup: Create file
    print(f"\n📝 Creating {filename}")
    with open(filename, "w") as f:
        f.write("test data")
    
    yield filename  # Provide filename to test
    
    # Teardown: Delete file
    print(f"\n🗑️ Deleting {filename}")
    if os.path.exists(filename):
        os.remove(filename)

def test_read_file(temp_file):
    """Test reading from file."""
    print(f"  📖 Reading {temp_file}")
    with open(temp_file, "r") as f:
        content = f.read()
    assert content == "test data"

def test_file_exists(temp_file):
    """Test file exists."""
    print(f"  🔍 Checking if {temp_file} exists")
    assert os.path.exists(temp_file)

**Each test:**
1. Creates the file (setup)
2. Runs the test
3. Deletes the file (teardown)

No leftover test files! 🎉

---

## Practical Examples

### Example 1: Shopping Cart Tests

In [ ]:
# shopping_cart.py
class ShoppingCart:
    def __init__(self):
        self.items = []
    
    def add_item(self, item, price):
        self.items.append({"item": item, "price": price})
    
    def total(self):
        return sum(item["price"] for item in self.items)
    
    def item_count(self):
        return len(self.items)
    
    def clear(self):
        self.items = []

In [ ]:
# test_shopping_cart.py
from shopping_cart import ShoppingCart

@pytest.fixture
def empty_cart():
    """Create an empty shopping cart."""
    return ShoppingCart()

@pytest.fixture
def cart_with_items():
    """Create a cart with sample items."""
    cart = ShoppingCart()
    cart.add_item("Book", 15.99)
    cart.add_item("Pen", 2.50)
    cart.add_item("Notebook", 5.99)
    return cart

def test_empty_cart_total(empty_cart):
    """Test empty cart has zero total."""
    assert empty_cart.total() == 0
    assert empty_cart.item_count() == 0

def test_add_item(empty_cart):
    """Test adding item to cart."""
    empty_cart.add_item("Book", 15.99)
    assert empty_cart.item_count() == 1
    assert empty_cart.total() == 15.99

def test_cart_total(cart_with_items):
    """Test calculating total."""
    expected = 15.99 + 2.50 + 5.99
    assert cart_with_items.total() == expected

def test_cart_item_count(cart_with_items):
    """Test item count."""
    assert cart_with_items.item_count() == 3

def test_clear_cart(cart_with_items):
    """Test clearing cart."""
    cart_with_items.clear()
    assert cart_with_items.item_count() == 0
    assert cart_with_items.total() == 0

---

## Best Practices

### 1. Fixture Naming

**✅ Good - Descriptive names:**
```python
@pytest.fixture
def empty_database():
    ...

@pytest.fixture
def user_with_admin_role():
    ...

@pytest.fixture
def temp_directory():
    ...
```

**❌ Bad - Vague names:**
```python
@pytest.fixture
def data():
    ...

@pytest.fixture
def obj():
    ...
```

### 2. Keep Fixtures Simple

**✅ Good - Focused fixture:**
```python
@pytest.fixture
def user():
    return {"name": "Alice", "age": 25}
```

**❌ Bad - Too much logic:**
```python
@pytest.fixture
def complex_setup():
    # 50 lines of setup code
    # Reading files
    # Making API calls
    # Complex calculations
    ...
```

### 3. Clean Up Resources

**Always clean up in fixtures that create resources:**

In [ ]:
@pytest.fixture
def resource():
    # Create resource
    resource = create_resource()
    
    yield resource
    
    # Always clean up
    resource.close()
    cleanup()

---

## Summary

### What We Learned:

✅ **Fixtures** - Reusable setup code for tests  
✅ **@pytest.fixture** - Decorator to create fixtures  
✅ **Using fixtures** - Add as test function parameters
```

### Next Steps:

Next, we'll learn about **parametrization** - how to run the same test with different inputs!